In [1]:
# Get raw data
with open('input/18.txt', 'r') as f:
    rawinput = f.read().strip()

In [2]:
# Part 1
conv = lambda x: int(x) if x.isnumeric() else -int(x[1:]) if x[0]=='-' and x[1:].isnumeric() else x
instr = [[i.split()[0], 
          conv(i.split()[1]),
          None if len(i.split())<3 else conv(i.split()[2])] 
         for i in rawinput.split('\n')]
reg_init = {k:0 for k in sorted({j for i in instr for j in i[1:] if isinstance(j, str)})}

def run(registers):
    ptr = 0
    lastsound = None
    while 0 <= ptr < len(instr):
        i, p1, p2 = instr[ptr]
        if isinstance(p2, str): p2 = registers[p2]
        if i=='snd':
            lastsound = registers[p1]
            ptr += 1
        elif i=='set':
            registers[p1] = p2
            ptr += 1
        elif i=='add':
            registers[p1] += p2
            ptr += 1
        elif i=='mul':
            registers[p1] *= p2
            ptr += 1
        elif i=='mod':
            registers[p1] %= p2
            ptr += 1
        elif i=='rcv':
            if registers[p1] != 0:
                return lastsound
            ptr += 1
        elif i=='jgz':
            ptr += (p2 if registers[p1] else 1)
    return 0

run(reg_init)

8600

In [3]:
# Part 2
reg_init = {p: {**{k:0 for k in sorted({j for i in instr for j in i[1:] if isinstance(j, str)})}, 
                **{'p': p}}
            for p in [0,1]}

def run(registers):
    ptr = [0, 0]
    queue = [[], []]
    send_count = [0, 0]
    while any([instr[ptr[i]][0] != 'rcv' or len(queue[i])>0 for i in [0,1]]):
        for p in [0,1]:
            #print(instr[ptr[p]])
            i, p1, p2 = instr[ptr[p]]
            if isinstance(p2, str): p2 = registers[p][p2]
            if i=='snd':
                queue[1-p] += [registers[p][p1]]
                send_count[p] += 1
                ptr[p] += 1
            elif i=='set':
                registers[p][p1] = p2
                ptr[p] += 1
            elif i=='add':
                registers[p][p1] += p2
                ptr[p] += 1
            elif i=='mul':
                registers[p][p1] *= p2
                ptr[p] += 1
            elif i=='mod':
                registers[p][p1] %= p2
                ptr[p] += 1
            elif i=='rcv':
                if len(queue[p]) > 0:
                    registers[p][p1] = queue[p].pop(0)
                    ptr[p] += 1
            elif i=='jgz':
                if isinstance(p1, str):  p1 = registers[p][p1]
                if p1 > 0:
                    ptr[p] += p2
                else:
                    ptr[p] += 1
    return send_count[1]

run(reg_init)

7239